<a href="https://colab.research.google.com/github/Pretzel-Solution/directional-sound-simulator/blob/master/simulate_directional_sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sofasonix
# press "Mount Drive"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from SOFASonix import SOFAFile
import scipy.io.wavfile as wav


In [3]:
# replace with your path!
loadsofa = SOFAFile.load('/content/drive/My Drive/directional_sound/HRIR_FULL2DEG.sofa')
data = loadsofa.data_ir
direction = loadsofa.SourcePosition
direction = direction[:,0:2] # the first two colums are azimuth and elevation angles in degree
sr = int(loadsofa.Data_SamplingRate[0]) # sampling_rate in Hz
#loadsofa.view() # if interested, you can explore the whole dataset

## create noise signal
duration = 0.5 #seconds
sample_n = int(duration*sr)
noise = np.random.uniform(-1,1,sample_n)

## create speech signal
# You can take the 'hallo2.wav' file from the repository or record your own voice e.g. with Audacity and sampling rate of 48kHz
load_speech = wav.read('/content/drive/My Drive/directional_sound/hallo2.wav')
speech = load_speech[1]
sampling_rate = load_speech[0]
if sampling_rate != sr:
  print('Warning: sampling_rate != sr')

Setting dimension 'r' from parameter 'ReceiverPosition'
Setting dimension 'e' from parameter 'EmitterPosition'
Setting dimension 'm' from parameter 'Data.IR'
Setting dimension 'n' from parameter 'Data.IR'
Inserting foreign parameter: 'GLOBAL:Author'
Inserting foreign parameter: 'GLOBAL:ListenerDescription'
Inserting foreign parameter: 'GLOBAL:ReceiverDescription'
Inserting foreign parameter: 'GLOBAL:SourceDescription'
Inserting foreign parameter: 'GLOBAL:RoomDescription'


In [5]:
def get_hrir(az_wish, el_wish):
  m_altered = np.abs(direction[:,0]- az_wish) + np.abs(direction[:,1]- el_wish)
  m_min = np.amin(m_altered, axis=0)
  i_row = np.argwhere(m_altered == m_min)[0][0]
  return data[i_row][0], data[i_row][1], i_row

In [6]:
def get_stereo(signal, az_wish, el_wish):
  '''
    signal:  numpy 1D array, e.g. signal=noise or signal=speech
    az_wish: azimuth angle in degree at which sound should be virtually placed
    el_wish: elevation angle in degree at which sound should be virtually placed
  '''
  hrir_l, hrir_r, i_row = get_hrir(az_wish, el_wish)
  left = np.convolve(signal, hrir_l, mode='valid') # 'valid': avoid boundary effects; The convolution product is only given for points where the signals overlap completely
  right = np.convolve(signal, hrir_r, mode='valid')
  audio = np.hstack((left.reshape(-1,1), right.reshape(-1,1)))
  scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
  file_name = 'stereo['+str(direction[i_row][0].round(1))+', '+str(direction[i_row][1].round(1))+'].wav'
  wav.write('/content/drive/My Drive/directional_sound/'+file_name, sr, scaled)

In [7]:
# az (azimuth) is the angle lying in the horizontal plane. It goes from 0° - nose direction, to 90° - left ear, to 180° - back, to 270° - right ear, and again to 360° - nose direction
# el (elevation) is the angle lying in the vertical plane. It goes from -88° - down, to 88° - up.
get_stereo(signal=speech, az_wish=90, el_wish=0)
# You have to use ear phones! Loudspeaker will not make the illusion of sound coming from a certain direction.

In [10]:
def get_roundabout(signal, az_begin, az_end, step_size, el=0):
  '''
    signal:   numpy 1D array, e.g. signal=noise or signal=speech
    az_begin: azimuth angle in degree at which the roundabout starts
    az_end:   azimuth angle in degree at which the roundabout ends
    step_size:azimuth angle step in degree
    el:       elevation in degree at which the sound travels horizontally around the head
  '''
  hrir_l, hrir_r, i_row = get_hrir(0, el_wish=0)
  left = np.convolve(signal, hrir_l, mode='valid')
  right = np.convolve(signal, hrir_r, mode='valid')
  audio = np.hstack((left.reshape(-1,1),right.reshape(-1,1)))
  print('Simulated alzimuth angles: ', np.arange(az_begin, az_end+1, step_size))
  for az_wish in np.arange(az_begin, az_end+1, step_size):
    hrir_l, hrir_r, i_row = get_hrir(az_wish, el_wish=el)
    left = np.convolve(signal, hrir_l, mode='valid')
    right = np.convolve(signal, hrir_r, mode='valid')
    audio_n = np.hstack((left.reshape(-1,1),right.reshape(-1,1)))
    audio = np.vstack((audio,audio_n))
  scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
  file_name = 'new_audio_roundabout_hallo.wav'
  wav.write('/content/drive/My Drive/directional_sound/'+file_name, sr, scaled)

In [12]:
get_roundabout(signal=speech, az_begin=0, az_end=360, step_size=45, el=0)

Simulated alzimuth angles:  [  0  45  90 135 180 225 270 315 360]
